In [1]:

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         376 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-cert

In [2]:
with open ('nyu-2451-34572-geojson.json') as json_data:
    new_york_data = json.load(json_data)

In [3]:
neighborhoods_data = new_york_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpHical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpHical coordinate of New York City are 40.7127281, -74.0060152.


In [6]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3.5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## USING BROOKLYN AS MY CHOICE OF NEIGHBORHOOD TO EXPLORE

In [7]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [8]:
brooklyn_data.loc[2, 'Neighborhood']

'Sunset Park'

In [9]:
#getting the longitude and latitude of Sunset park as i want to explore locations and venues close to it.

neighborhood_latitude = brooklyn_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Sunset Park are 40.64510294925429, -74.01031618527784.


In [10]:

CLIENT_ID = 'O3X2YPNHOZWVQKI5NSN5UDRNQQYJLAACZJUCWS5MIEMFVGUJ' # your Foursquare ID
CLIENT_SECRET = 'HCT1WNBGLIMXVV1CEUYMP41DQUS5GRZLPPGXHDHAU1BLKEK3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + 'O3X2YPNHOZWVQKI5NSN5UDRNQQYJLAACZJUCWS5MIEMFVGUJ')
print('CLIENT_SECRET:' + 'RLDSLYXBS1K0LFZA4GRYIOTO3RUP5OAOR2BEMQ2A3QGPTOVB')

Your credentials:
CLIENT_ID: O3X2YPNHOZWVQKI5NSN5UDRNQQYJLAACZJUCWS5MIEMFVGUJ
CLIENT_SECRET:RLDSLYXBS1K0LFZA4GRYIOTO3RUP5OAOR2BEMQ2A3QGPTOVB


In [11]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=O3X2YPNHOZWVQKI5NSN5UDRNQQYJLAACZJUCWS5MIEMFVGUJ&client_secret=HCT1WNBGLIMXVV1CEUYMP41DQUS5GRZLPPGXHDHAU1BLKEK3&v=20180605&ll=40.64510294925429,-74.01031618527784&radius=500&limit=100'

In [12]:
results = requests.get(url).json()

In [13]:
#A function that generates neighborhoods in  New york
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
            
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
              v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=O3X2YPNHOZWVQKI5NSN5UDRNQQYJLAACZJUCWS5MIEMFVGUJ&client_secret=HCT1WNBGLIMXVV1CEUYMP41DQUS5GRZLPPGXHDHAU1BLKEK3&v=20180605&ll=40.625801065010656,-74.03062069353813&radius=1000&limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1a04ec4518>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
#putting the neighborhoods into a dataframe 
a =